In [1]:
import pandas as pd
import numpy as np
import torch
import utils.print as print_f
import os

from utils.engine import evaluate, get_iou_types
from utils.plot import plot_losses, plot_performance
from data.strs import TaskStrs, SourceStrs, FusionStrs

from models.utils import get_model_size_in_MB
from models.build import create_model_from_setup
from models.setup import ModelSetup
from models.train import TrainingInfo
from utils.save import check_best, end_train, get_data_from_metric_logger
from data.load import get_datasets, get_dataloaders
from IPython.display import clear_output
from utils.eval import get_ap_ar, get_ap_ar_for_train_val, get_performance
from utils.train import get_optimiser, get_lr_scheduler, print_params_setup, get_coco_eval_params, get_dynamic_loss
from utils.init import reproducibility, clean_memory_get_device
from data.paths import MIMIC_EYE_PATH
# from datetime import datetime
# from models.dynamic_loss import DynamicWeightedLoss
from utils.engine import train_one_epoch, evaluate
from torch import optim
from models.setup import ModelSetup
from data.paths import MIMIC_EYE_PATH
# from data.load import seed_worker, get_dataloader_g

## Suppress the assignement warning from pandas.r
pd.options.mode.chained_assignment = None  # default='warn'

## Supress user warning
import warnings
warnings.filterwarnings("ignore")

from models.components.task_performers import ObjectDetectionPerformer
from data.constants import DEFAULT_MIMIC_CLINICAL_NUM_COLS, DEFAULT_MIMIC_CLINICAL_CAT_COLS
from sklearn.metrics import roc_auc_score

%matplotlib inline

In [2]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"
# 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

In [3]:
device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CPU]


In [4]:
if torch.cuda.is_available():
    print(torch.cuda.memory_summary())

In [5]:
common_args = {
    # "sources": [SourceStrs.XRAYS, SourceStrs.CLINICAL,],
    # "tasks": [
    #     # TaskStrs.LESION_DETECTION,
    #     TaskStrs.FIXATION_GENERATION,
    #     TaskStrs.CHEXPERT_CLASSIFICATION,
    #     TaskStrs.NEGBIO_CLASSIFICATION,
    # ],
    "performance_standard_task": TaskStrs.CHEXPERT_CLASSIFICATION,
    "performance_standard_metric": "auc",

    # "performance_standard_task": TaskStrs.LESION_DETECTION,
    # "performance_standard_metric": "ap",

    "decoder_channels": [128, 64, 32, 16, 8],
    "optimiser": "sgd",
    "lr": 1e-3,  # should multiply by 8 or just use 1e-2, to test if it will explode.
    "batch_size": 16,
    "weight_decay": 1e-9,  # 1E-5
    # "pretrained": True,
    "record_training_performance": True,
    "image_size": 512,
    "warmup_epochs": 0,
    "lr_scheduler": "ReduceLROnPlateau",
    "reduceLROnPlateau_factor": 0.1,
    "reduceLROnPlateau_patience": 999,
    "reduceLROnPlateau_full_stop": True,
    "multiStepLR_milestones": 100,
    "multiStepLR_gamma": 0.1,
    "use_mask": False,
    "gt_in_train_till": 0,
    "box_head_dropout_rate": 0,
    "measure_test": True,
    "fiaxtions_mode": "reporting",
}

with_fix_args= {
    "sources": [SourceStrs.XRAYS],
    # "sources": [SourceStrs.XRAYS, SourceStrs.CLINICAL,],
    "tasks": [
        # TaskStrs.LESION_DETECTION,
        TaskStrs.FIXATION_GENERATION,
        TaskStrs.CHEXPERT_CLASSIFICATION,
        # TaskStrs.NEGBIO_CLASSIFICATION,
    ],
}

without_fix_args= {
    "sources": [SourceStrs.XRAYS],
    # "sources": [SourceStrs.XRAYS, SourceStrs.CLINICAL,],
    "tasks": [
        # TaskStrs.LESION_DETECTION,
        # TaskStrs.FIXATION_GENERATION,
        TaskStrs.CHEXPERT_CLASSIFICATION,
        # TaskStrs.NEGBIO_CLASSIFICATION,
    ],
}

small_model_args = {
    "mask_hidden_layers": 64,
    "fuse_conv_channels": 64,
    "representation_size": 64,  # 32
    # "clinical_input_channels": 64,
    # "clinical_conv_channels": 64,
    # "clinical_expand_conv_channels": 64,
    "backbone_out_channels": 64,
}

mobilenet_args = {
    "backbone": "mobilenet_v3",
    "using_fpn": False,
}


In [6]:
all_model_setups = [
    ModelSetup(
        name="chexpert_with_fix",
        **mobilenet_args,
        **small_model_args,
        **common_args,
        **with_fix_args
    ),
    # ModelSetup(
    #     name="chexpert_without_fix",
    #     **mobilenet_args,
    #     **small_model_args,
    #     **common_args,
    #     **without_fix_args
    # )
]

In [7]:
num_epochs = 1
train_infos = []

In [8]:
# we have to mention that in order to provide objective evaluation, we compromise the dataset size, which also nagatively afftect the performance and generlaization.
for setup in all_model_setups:
    assert (
        len(setup.sources) > 0 and len(setup.tasks) > 0
    ), "Need at least one source and task."

    print_f.print_title("Preparing for the training.")

    train_info = TrainingInfo(setup)

    model = create_model_from_setup(setup=setup)
    model = model.to(device)

    train_info.all_tasks = list(model.task_performers.keys())
    print(f" Model Size [{get_model_size_in_MB(model):.3f}] MB")

    ################ Datasets ################
    dataset_params_dict = {
        "MIMIC_EYE_PATH": MIMIC_EYE_PATH,
        "labels_cols": setup.lesion_label_cols,
        "with_clincal": SourceStrs.CLINICAL in setup.sources,
        "with_xrays": SourceStrs.XRAYS in setup.sources,
        "with_bboxes": TaskStrs.LESION_DETECTION in setup.tasks,
        "with_fixations": TaskStrs.FIXATION_GENERATION in setup.tasks,
        "with_chexpert": TaskStrs.CHEXPERT_CLASSIFICATION in setup.tasks,
        "with_negbio": TaskStrs.NEGBIO_CLASSIFICATION in setup.tasks,
        "fiaxtions_mode": setup.fiaxtions_mode,
        "clinical_numerical_cols": setup.clinical_num,
        "clinical_categorical_cols": setup.clinical_cat,
    }

    detect_eval_dataset, train_dataset, val_dataset, test_dataset = get_datasets(
        dataset_params_dict=dataset_params_dict,
    )

    train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
        train_dataset, val_dataset, test_dataset, batch_size=setup.batch_size,
    )

    train_coco = None
    val_coco = None
    test_coco = None
    eval_params_dict = None

    if TaskStrs.LESION_DETECTION in setup.tasks:
        train_coco, val_coco, test_coco, eval_params_dict = get_coco_eval_params(
            source_name=SourceStrs.XRAYS,
            task_name=TaskStrs.LESION_DETECTION,
            train_dataloader=train_dataloader,
            val_dataloader=val_dataloader,
            test_dataloader=test_dataloader,
            detect_eval_dataset=detect_eval_dataset,
            iou_thrs=setup.iou_thrs,
            use_iobb=setup.use_iobb,
        )

    train_info.timer.start_training()

    dynamic_loss_weight = get_dynamic_loss(
        loss_keys=model.get_all_losses_keys(), device=device
    )
    params = model.get_all_params(dynamic_loss_weight=dynamic_loss_weight)

    iou_types = get_iou_types(model, setup)
    optimizer = get_optimiser(params, setup)
    lr_scheduler = get_lr_scheduler(optimizer, setup)

    print_f.print_title(
        f"Start training. Preparing Took [{train_info.timer.has_took_sec_from_init()}] sec"
    )

    train_info.timer.start_training()

    val_loss = None

    # Start the training from here.
    for e in range(1, num_epochs + 1):

        print_f.print_title(f"Training model: [{setup.name}]")

        print(train_info)

        train_info.epoch = e

        train_info.timer.start_epoch()

        if (
            any(
                [
                    isinstance(p, ObjectDetectionPerformer)
                    for p in model.task_performers.values()
                ]
            )
            and train_info.epoch > setup.gt_in_train_till
        ):
            model.task_performers[
                TaskStrs.LESION_DETECTION
            ].roi_heads.use_gt_in_train = False

        ###### Perform training and show the training result here ######
        model.train()

        train_info.last_train_evaluator, train_loger = train_one_epoch(
            setup=setup,
            model=model,
            optimizer=optimizer,
            # data_loader=train_dataloader,
            data_loader=test_dataloader,
            device=device,
            epoch=train_info.epoch,
            print_freq=10,
            iou_types=iou_types,
            coco=train_coco,
            score_thres=None,
            evaluate_on_run=True,
            params_dict=eval_params_dict,
            dynamic_loss_weight=dynamic_loss_weight,
        )

        # train_info.train_evaluators.append(train_evaluator)
        train_info.train_losses.append(get_data_from_metric_logger(train_loger))
        ################################################################

        ####### Put the model into evaluation mode, start evaluating the current model #######
        model.eval()

        train_info.last_val_evaluator, val_logger = evaluate(
            setup=setup,
            model=model,
            data_loader=val_dataloader,
            device=device,
            params_dict=eval_params_dict,
            coco=val_coco,
            iou_types=iou_types,
            score_thres=None,
        )

        # train_info.val_evaluators.append(val_evaluator)
        train_info.val_losses.append(get_data_from_metric_logger(val_logger))

        train_info.performance["train"].append(
            get_performance(
                train_info.all_tasks,
                train_info.last_train_evaluator,
                iouThr=0.5,
                areaRng="all",
                maxDets=10,
            )
        )

        train_info.performance["val"].append(
            get_performance(
                train_info.all_tasks,
                train_info.last_val_evaluator,
                iouThr=0.5,
                areaRng="all",
                maxDets=10,
            )
        )

        if setup.measure_test:
            train_info.test_evaluator, test_logger = evaluate(
                setup=setup,
                model=model,
                data_loader=test_dataloader,
                device=device,
                params_dict=eval_params_dict,
                coco=test_coco,
                iou_types=iou_types,
                score_thres=None,
            )
            train_info.test_losses.append(get_data_from_metric_logger(test_logger))

            train_info.performance["test"].append(
                get_performance(
                    train_info.all_tasks,
                    train_info.test_evaluator,
                    iouThr=0.5,
                    areaRng="all",
                    maxDets=10,
                )
            )

        # update the learning rate

        val_loss = train_info.val_losses[-1]["loss"]

        epoch_took, sec_already_took, speed = train_info.timer.end_epoch(
            train_info.epoch
        )

        if train_info.epoch > setup.warmup_epochs:
            if not lr_scheduler is None:
                if isinstance(lr_scheduler, optim.lr_scheduler.ReduceLROnPlateau):
                    if (
                        setup.reduceLROnPlateau_full_stop
                        and lr_scheduler.num_bad_epochs
                        >= setup.reduceLROnPlateau_patience
                    ):
                        print_f.print_title(
                            f"| EarlyStop | Epoch [{train_info.epoch}] Done | It has took [{sec_already_took/60:.2f}] min, Avg time: [{speed:.2f}] sec/epoch | Estimate time for [{num_epochs}] epochs: [{speed*num_epochs/60:.2f}] min | Epoch took [{epoch_took.seconds}] sec |"
                        )
                        break
                    lr_scheduler.step(val_loss)
                else:
                    lr_scheduler.step()

        # Clean everything before we show the evaluating result in this stage, so we can inspect the training progress.
        clear_output()

        # if model_setup.record_training_performance:
        # plot all matrics
        plot_performance(
            performance=train_info.performance,
            all_tasks=train_info.all_tasks,
            fig_title="Performance",
        )

        plot_losses(
            train_info.train_losses,
            train_info.val_losses,
            test_logers=train_info.test_losses,
        )

        print_str = f"| Epoch [{train_info.epoch}] Done | It has took [{sec_already_took/60:.2f}] min, Avg time: [{speed:.2f}] sec/epoch | Estimate time for [{num_epochs}] epochs: [{speed*num_epochs/60:.2f}] min | Epoch took [{epoch_took.seconds}] sec | "

        if lr_scheduler and hasattr(lr_scheduler, "num_bad_epochs"):
            print_str += f"Patience [{lr_scheduler.num_bad_epochs}] |"

        print_f.print_title(print_str)

        train_info

        #######################################################################################
        if setup.save_early_stop_model:
            val_performance_value, train_info = check_best(
                setup=setup,
                val_performance_value=train_info.performance["val"][-1][
                    setup.performance_standard_task
                ][setup.performance_standard_metric],
                device=device,
                eval_params_dict=eval_params_dict,
                train_info=train_info,
                model=model,
                optim=optimizer,
                test_dataloader=test_dataloader,
                test_coco=test_coco,
                iou_types=iou_types,
                score_thres=None,
                dynamic_weight=dynamic_loss_weight,
            )

    train_info = end_train(
        setup=setup,
        train_info=train_info,
        model=model,
        optim=optimizer,
        eval_params_dict=eval_params_dict,
        last_val_performance=train_info.performance["val"][-1][
            setup.performance_standard_task
        ][setup.performance_standard_metric],
        test_dataloader=test_dataloader,
        device=device,
        test_coco=test_coco,
        iou_types=iou_types,
        score_thres=None,
        dynamic_weight=dynamic_loss_weight,
    )

    train_infos.append(train_info)


# TODO: allow for any kind of pipeline.


====================Preparing for the training.====================
Using pretrained backbone. mobilenet_v3
 Model Size [6.266] MB
Using SGD as optimizer with lr=0.001
====================Start training. Preparing Took [0] sec====================
====================Training model: [chexpert_with_fix]====================
========================================For Training [chexpert_with_fix]========================================
ModelSetup(name='chexpert_with_fix', sources=['xrays'], tasks=['fixation-generation', 'chexpert-classification'], fusor='element-wise sum', decoder_channels=[128, 64, 32, 16, 8], lesion_label_cols=['Pulmonary edema', 'Enlarged cardiac silhouette', 'Consolidation', 'Atelectasis', 'Pleural abnormality'], save_early_stop_model=True, record_training_performance=True, backbone='mobilenet_v3', optimiser='sgd', lr=0.001, weight_decay=1e-09, image_backbone_pretrained=True, heatmap_backbone_pretrained=False, image_size=512, backbone_out_channels=64, batch_size=16, wa

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [ ]:
train_info.last_train_evaluator['fixation-generation'].ious

[0.00067901611328125,
 0.0033111572265625,
 0.000255584716796875,
 0.00171661376953125,
 0.0,
 0.005031585693359375,
 0.0067138671875,
 0.0,
 0.000133514404296875,
 3.4332275390625e-05,
 0.000637054443359375,
 0.0015411376953125,
 0.0,
 0.00139617919921875,
 0.0,
 0.0,
 0.0,
 0.00043487548828125,
 0.000682830810546875,
 0.000141143798828125,
 0.00478363037109375,
 0.0,
 0.005218505859375,
 0.003734588623046875,
 0.0,
 0.000354766845703125,
 0.0,
 0.00022125244140625,
 0.0,
 0.00397491455078125,
 0.000225067138671875,
 0.0,
 0.0,
 0.004596710205078125,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.00074005126953125,
 0.004131317138671875,
 0.0,
 0.0,
 0.001247406005859375,
 0.00201416015625,
 0.002574920654296875,
 0.0,
 0.0,
 0.000591278076171875,
 0.000179290771484375,
 0.0,
 0.0,
 0.0,
 0.00058746337890625,
 0.000453948974609375,
 0.0013275146484375,
 0.0,
 0.001773834228515625,
 0.000652313232421875,
 0.00023651123046875,
 0.00029754638671875,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0057144165039

In [ ]:
def get_iou(preds, gts):
    intersection = np.logical_and(gts, preds)
    union = np.logical_or(gts, preds)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score 
    

In [ ]:
get_iou(train_info.last_train_evaluator['fixation-generation'].gts[0], train_info.last_train_evaluator['fixation-generation'].preds[0])

# did I put the relu at the last layer for the decovder part? 

In [ ]:
train_info.last_train_evaluator['fixation-generation'].gts[0].dtype

dtype('uint8')

In [ ]:
train_info.last_train_evaluator['fixation-generation'].preds[0].dtype

dtype('float32')

In [ ]:
for train_info in train_infos:
    print(train_info)
    plot_performance(
        performance=train_info.performance,
        all_tasks=train_info.all_tasks,
        fig_title="Performance",
    )

    plot_losses(
        train_info.train_losses,
        train_info.val_losses,
        test_logers=train_info.test_losses,
    )



NameError: name 'train_infos' is not defined